## GPT from scratch based on the Transformer paper

This model is transformer's decoder only model. 


In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
# Get Data 
import urllib.request
txt_url = "https://s3.amazonaws.com/text-datasets/nietzsche.txt"
#txt_url = "https://raw.githubusercontent.com/ravexina/shakespeare-plays-dataset-scraper/master/shakespeare-db/All's%20Well%20That%20Ends%20Well.txt"

!wget "{txt_url}" -O train.txt

--2023-07-26 14:18:36--  https://s3.amazonaws.com/text-datasets/nietzsche.txt
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.89.174, 52.217.173.136, 52.217.142.176, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.89.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 600901 (587K) [text/plain]
Saving to: ‘train.txt’

train.txt           100%[===================>] 586.82K   816KB/s    in 0.7s    

2023-07-26 14:18:37 (816 KB/s) - ‘train.txt’ saved [600901/600901]



In [3]:
# hyperparameters
batch_size = 16 # number of independent sequences processed in parallel 
block_size = 32 # number of maximum context length for predictions
max_iters = 30000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'mps' if torch.backends.mps.is_available() else 'cpu'
eval_iters = 200
n_embed = 64
n_head = 4 # number of head for multi-heads (headの数)
n_layer = 4 # number of multi-head layers (multi-headのレイヤーの数)
dropout = 0.0
# ------------

print(device)

mps


In [4]:
# Tokenize
class Tokenizer():
    def __init__(self, characters_list:list) -> None:
        self.characters_list = characters_list

    # map string to int
    def mapStringToInt(self):
        self.MapStrToInt = {ch:i for i, ch in enumerate(self.characters_list)}
        return self.MapStrToInt

    # map to int to string
    def mapIntToString(self):
        self.MapIntToStr = {i:ch for i, ch in enumerate(self.characters_list)}
        return self.MapIntToStr

    # string to int
    def encoder(self, string:str):
        CharMaps=self.MapStrToInt
        return [CharMaps[c] for c in string]

    # int to string
    def decoder(self, int_list:list):
        CharMaps=self.MapIntToStr
        return ''.join([CharMaps[i] for i in int_list])


In [5]:
# Training and validate
class DataLoader():
    def __init__(self, data, block_size, batch_size) -> None:
        self.data = data
        self.block_size = block_size
        self.batch_size = batch_size

    # split data into validation and training data.
    def splitData(self, splitPercentage=0.9):
        n = int(splitPercentage*len(self.data))
        self.val_data = self.data[n:]
        self.train_data = self.data[:n]

        return self.train_data, self.val_data

    # Get chunk of data
    def getBlock(self, start, end, data_list:list):
        return data_list[start:end]

    # get batch
    def get_batch(self, split):
        # choose train_data or val_data
        data = self.train_data if split == 'train' else self.val_data
        # generate batch size random numbers
        ix = torch.randint(len(data) - self.block_size, (self.batch_size,))
        # first block size words
        # stack to create new dimension
        x = torch.stack([self.getBlock(start=i, end=i+self.block_size, data_list=self.data) for i in ix])
        # offset of x
        y = torch.stack([self.getBlock(start=i+1, end=i+self.block_size+1, data_list=self.data) for i in ix])
        x, y = x.to(device), y.to(device)
        return x, y

In [6]:
with open("train.txt", "r", encoding='utf-8') as f:
    text = f.read()

print(f"Len(train.txt): {len(text)}")

Len(train.txt): 600893


In [7]:
# characters inside the training data.
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)
print(chars)


 !"'(),-.0123456789:;=?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzÆäæéë
84
['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Æ', 'ä', 'æ', 'é', 'ë']


In [8]:
tokenizer = Tokenizer(characters_list = chars)

MapStrToInt = tokenizer.mapStringToInt()
MapIntToStr = tokenizer.mapIntToString()

print(tokenizer.encoder(string="hi there"))
print(tokenizer.decoder(int_list = tokenizer.encoder(string="hi there")))

[60, 61, 1, 72, 60, 57, 70, 57]
hi there


In [9]:
# Encode the train.txt and wrap it in tensor
data = torch.tensor(tokenizer.encoder(text), dtype=torch.long)
print(f"data.shape: {data.shape}")

data.shape: torch.Size([600893])


In [10]:
dataLoader = DataLoader(data = data,
                        block_size= block_size,
                        batch_size= batch_size)

In [11]:
# Split the data
train_data, val_data = dataLoader.splitData()

In [12]:
# Split the data to chunk
# maximum context length for predictions

x = dataLoader.getBlock(data_list=train_data, start=0, end=block_size)
y = dataLoader.getBlock(data_list=train_data, start=1, end=block_size+1)

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When input is {context} the target is {target}")

When input is tensor([39]) the target is 41
When input is tensor([39, 41]) the target is 28
When input is tensor([39, 41, 28]) the target is 29
When input is tensor([39, 41, 28, 29]) the target is 24
When input is tensor([39, 41, 28, 29, 24]) the target is 26
When input is tensor([39, 41, 28, 29, 24, 26]) the target is 28
When input is tensor([39, 41, 28, 29, 24, 26, 28]) the target is 0
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0]) the target is 0
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0]) the target is 0
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0]) the target is 42
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0, 42]) the target is 44
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0, 42, 44]) the target is 39
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0, 42, 44, 39]) the target is 39
When input is tensor([39, 41, 28, 29, 24, 26, 28,  0,  0,  0, 42, 44, 39, 39]) the target is 38
When input is tensor([3

In [13]:
# generate random numbers and seed it so the numbers wont change every time we execute
torch.manual_seed(1337)

# number of independent sequences for parallel process
xb, yb = dataLoader.get_batch(split='train')
print("inputs:")
print(xb.shape)
print(xb)

print("targets:")
print(yb.shape)
print(yb)

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b:t+1]
        target = yb[b,t]
        print(f"When input is {context.tolist()} the target is {target}")

inputs:
torch.Size([16, 32])
tensor([[56,  1,  3, 57, 58, 58, 57, 55, 72,  7,  3,  1, 53, 71,  1, 72, 60, 57,
          1, 66, 53, 72, 73, 70, 53, 64,  0, 68, 60, 61, 64, 67],
        [67, 72, 60, 61, 66, 59,  1, 61, 71,  1, 65, 67, 70, 57,  1, 55, 67, 65,
         65, 67, 66,  1, 72, 60, 53, 66,  1, 72, 67,  1, 56, 67],
        [53, 72,  1, 61, 71,  1, 66, 67, 75,  1, 72, 60, 57,  1, 65, 67, 70, 53,
         64,  1, 68, 57, 70, 71, 68, 57, 55, 72, 61, 74, 57,  7],
        [74, 57, 71,  0, 61, 65, 68, 57, 70, 58, 57, 55, 72, 64, 77,  7,  1, 60,
         57, 71, 61, 72, 53, 72, 61, 66, 59, 64, 77,  7,  1, 53],
        [65, 67, 70, 57,  1, 53, 59, 70, 57, 57, 53, 54, 64, 57,  0, 75, 61, 72,
         60,  1, 72, 60, 57,  1, 64, 53, 68, 71, 57,  1, 67, 58],
        [60, 57,  1, 31, 67, 60, 57, 66, 71, 72, 53, 73, 58, 57, 66,  7,  1, 29,
         70, 57, 56, 57, 70, 61, 55, 63,  1, 72, 60, 57,  1, 42],
        [72, 53, 61, 66, 64, 77,  1, 54, 67, 53, 71, 72,  1, 67, 58,  1, 72, 60,
        

In [14]:
print(xb)

tensor([[56,  1,  3, 57, 58, 58, 57, 55, 72,  7,  3,  1, 53, 71,  1, 72, 60, 57,
          1, 66, 53, 72, 73, 70, 53, 64,  0, 68, 60, 61, 64, 67],
        [67, 72, 60, 61, 66, 59,  1, 61, 71,  1, 65, 67, 70, 57,  1, 55, 67, 65,
         65, 67, 66,  1, 72, 60, 53, 66,  1, 72, 67,  1, 56, 67],
        [53, 72,  1, 61, 71,  1, 66, 67, 75,  1, 72, 60, 57,  1, 65, 67, 70, 53,
         64,  1, 68, 57, 70, 71, 68, 57, 55, 72, 61, 74, 57,  7],
        [74, 57, 71,  0, 61, 65, 68, 57, 70, 58, 57, 55, 72, 64, 77,  7,  1, 60,
         57, 71, 61, 72, 53, 72, 61, 66, 59, 64, 77,  7,  1, 53],
        [65, 67, 70, 57,  1, 53, 59, 70, 57, 57, 53, 54, 64, 57,  0, 75, 61, 72,
         60,  1, 72, 60, 57,  1, 64, 53, 68, 71, 57,  1, 67, 58],
        [60, 57,  1, 31, 67, 60, 57, 66, 71, 72, 53, 73, 58, 57, 66,  7,  1, 29,
         70, 57, 56, 57, 70, 61, 55, 63,  1, 72, 60, 57,  1, 42],
        [72, 53, 61, 66, 64, 77,  1, 54, 67, 53, 71, 72,  1, 67, 58,  1, 72, 60,
         57,  1, 71, 60, 53, 70, 68, 

For example: <br>
For the first one (67), BigramLangModel will look for the 67th row of the Embedding table

最初の行（67）に対して、BigramLangModelは埋め込みテーブルの67行目を探す。

```
tensor([[67, 65,  0, 53, 54, 71, 67, 64],  
        [ 1,  1, 32,  1, 53, 65,  7,  1],  
        [53, 64,  1, 72, 60, 53, 72,  1],  
        [61, 67, 66,  1, 53, 66, 56,  1]])   
```

### Single Head 

Scaled Dot-Product Attention <br>
https://paperswithcode.com/method/scaled

Figure2 (Left) in the ["Attention Is All You Need" paper](https://arxiv.org/pdf/1706.03762v5.pdf)

![image.png](./img/ScaledDotProduct.png)


In [15]:
# Single Head
class Head(nn.Module):
    def __init__(self,  head_size):
        super().__init__()
        self.key = nn.Linear(n_embed, head_size, bias=False)
        self.query = nn.Linear(n_embed, head_size, bias=False)
        self.value = nn.Linear(n_embed, head_size, bias=False)
        self.register_buffer("tril", torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def ScaledDotProductAttention(self, k, q, B, T, C):
        # MatMul + Scale
        weight = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, head_size) @ (B, head_size, T) --> (B, T, T) 
        # Mask (opt.)
        weight = weight.masked_fill(self.tril[:T,:T] == 0, float('-inf'))  # (B, T, T) # setting it to infinity, we will not aggregate the tokens from the past, in other words future cannot communicate from the past
        # SoftMax
        weight = F.softmax(weight, dim = -1) # Normalize weight

        return weight

    def forward(self, x):
        B,T,C = x.shape

        k = self.key(x) # (B, T, head_size)
        q = self.query(x) # (B, T, head_size)

        # compute attention scores # Scale Dot-Product Attention
        weight = self.ScaledDotProductAttention(k, q, B, T, C)
        weight = self.dropout(weight)

        # perform the weighted aggregation of the value
        v = self.value(x) # Thing that get aggregated for the purposes of the single head
        xBagOfWords = weight @ v

        return xBagOfWords




![image.png](./img/MultiHeadAttention.png)

In [16]:
# multi heads of self attention in parallel
class MultiHeadAttention(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embed, n_embed)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # Scaled Dot Product Attention and concat
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        # Linear layer at the end
        out = self.proj(out)
        out = self.dropout(out)

        return out

## Blocks

Way to prevent degradation problem, by skipping some part of the layers.

Degradation problem: <br>
Shallower networks perform better than the deeper counterparts with few more layers added to them

https://arxiv.org/abs/1512.03385

https://paperswithcode.com/method/residual-block

https://towardsdatascience.com/residual-blocks-building-blocks-of-resnet-fd90ca15d6ec



In [17]:
# a simple linear layer followed by a non-linearity
class FeedFoward(nn.Module):
    def __init__(self, n_embed):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embed, 4 * n_embed),
            nn.ReLU(),
            nn.Linear(4 * n_embed, n_embed),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [18]:
# Transformer block:  communication followed by computation
class Block(nn.Module):
    def __init__(self, n_embed, n_head):
        #  n_embed : embedding demensions,
        #  n_head: the number of heads
        super().__init__()
        head_size = n_embed // n_head
        self.sa = MultiHeadAttention(num_heads = n_head, head_size = head_size)
        self.ffwd = FeedFoward(n_embed)
        self.ln1 = nn.LayerNorm(n_embed)
        self.ln2 = nn.LayerNorm(n_embed)

    def forward(self, x):
        '''
        Different from the "All You Need Is Attention" paper.
        In transformer paper, Norm Layer is located after the multihead attention but this time it is located after the multihead attention.
        '''
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))

        return x


In [19]:
class BigramLangModel(nn.Module):
    def __init__(self, vocab_size, n_embed) -> None:
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embed)
        self.position_embedding_table = nn.Embedding(block_size, n_embed) # Embedding table for position of the token
        self.blocks = nn.Sequential(*[Block(n_embed, n_head=n_head) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embed, vocab_size)
        self.ln_f = nn.LayerNorm(n_embed)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both tensor
        tok_emb = self.token_embedding_table(idx) # (Batch(batch size), Time(block size), Channel(embed))
        pos_emb = self.position_embedding_table(torch.arange(T,device=device)) # (T,C) integers of 0 ~ T-1
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # apply one head of self-attention (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (Batch(batch size), Time(block size), vocab size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # (Batch(batch size), Time(block size), Channel(embed))
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B,T) array of indices in current context
        for _ in range(max_new_tokens):
            # crop idx to the last block size tokens # we can never have more than block size
            idx_cond = idx[:,-block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last step
            logits = logits[:, -1, :]
            # apply softmax to the probabilities
            probs = F.softmax(logits, dim =-1) #(B,C)
            # sample from distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            #append sampled index to running sequence
            idx = torch.cat((idx, idx_next), dim = 1)
        return idx


In [20]:
@torch.no_grad()
def estimate_loss(model):
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = dataLoader.get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out


In [21]:
model = BigramLangModel(vocab_size = vocab_size, n_embed=n_embed)
model.to(device)


# print the number of parameters in the model
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss(model)
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = dataLoader.get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decoder(model.generate(context, max_new_tokens=2000)[0].tolist()))

0.21218 M parameters
step 0: train loss 4.6284, val loss 4.6227
step 100: train loss 2.7023, val loss 2.7118
step 200: train loss 2.5632, val loss 2.5833
step 300: train loss 2.4993, val loss 2.5075
step 400: train loss 2.4224, val loss 2.4341
step 500: train loss 2.3408, val loss 2.3758
step 600: train loss 2.2931, val loss 2.3097
step 700: train loss 2.2281, val loss 2.2496
step 800: train loss 2.1797, val loss 2.1975
step 900: train loss 2.1387, val loss 2.1641
step 1000: train loss 2.0957, val loss 2.1204
step 1100: train loss 2.0589, val loss 2.0803
step 1200: train loss 2.0248, val loss 2.0452
step 1300: train loss 2.0008, val loss 2.0134
step 1400: train loss 1.9672, val loss 1.9923
step 1500: train loss 1.9411, val loss 1.9590
step 1600: train loss 1.9240, val loss 1.9344
step 1700: train loss 1.9110, val loss 1.9205
step 1800: train loss 1.8964, val loss 1.8839
step 1900: train loss 1.8754, val loss 1.8789
step 2000: train loss 1.8549, val loss 1.8721
step 2100: train loss 1.8